In [0]:
from datetime import date, timedelta
import requests

current_date = date.fromisoformat('2024-01-01')
next_date = date.fromisoformat('2024-01-01')
end_date = date.fromisoformat('2024-02-01')
request_parameters = []
curr_request = []
ten_days = timedelta(days=10)
data_rows = []
column_names = None

while current_date <= end_date:
    next_date = current_date + ten_days
    curr_request = {"deliveryDateFrom": current_date.strftime("%Y-%m-%d"),
        "deliveryDateTo": next_date.strftime("%Y-%m-%d")}
    request_parameters.append(curr_request)
    current_date = next_date + timedelta(days=1)

url = "https://api.ercot.com/api/public-reports/np6-905-cd/spp_node_zone_hub?&settlementPoint=ANACACHO_ANA&deliveryIntervalFrom=1&deliveryIntervalTo=1"

payload = {}
files={}
headers = {
  'Ocp-Apim-Subscription-Key': '#redacted',
  'Authorization': 'Bearer #redacted',
  'Cookie': 'incap_ses_1539_3031165=ijKaQL5XnWUDmlqt0aBbFVgqGmkAAAAAz8KD0YkXVJBwRA6YmeTvEg==; nlbi_3031165=5GyibWI/h1hNDc5RYHvhKwAAAAD/ViRzRvCpFG8WDcWwwc7o; visid_incap_3031165=h7Qk2+dDTWC3ZfJn5AfnIYgYGmkAAAAAQUIPAAAAAABp/gnlBw6P56dO0pjT+Gnp'
}

for params in request_parameters:
    response = requests.request("GET", url, headers=headers, data=payload, files=files, params=params)
    if column_names is None:
        column_names = [field['name'] for field in response.json()['fields']]
    current_data_rows = response.json()['data']
    data_rows.extend(current_data_rows)

# Create Spark DataFrame
spark_df = spark.createDataFrame(data_rows, schema=column_names)

# Type conversion
from pyspark.sql.functions import col, to_date
spark_df = spark_df.withColumn("deliveryDate", to_date(col("deliveryDate")))
for col_name in ['deliveryHour', 'deliveryInterval', 'settlementPointPrice']:
    spark_df = spark_df.withColumn(col_name, col(col_name).cast("double"))

In [0]:
display(spark_df.show(5))

+------------+------------+----------------+---------------+-------------------+--------------------+-------+
|deliveryDate|deliveryHour|deliveryInterval|settlementPoint|settlementPointType|settlementPointPrice|DSTFlag|
+------------+------------+----------------+---------------+-------------------+--------------------+-------+
|  2024-01-11|         1.0|             1.0|   ANACACHO_ANA|                 RN|              142.36|  false|
|  2024-01-11|         2.0|             1.0|   ANACACHO_ANA|                 RN|              243.04|  false|
|  2024-01-11|         3.0|             1.0|   ANACACHO_ANA|                 RN|                89.6|  false|
|  2024-01-11|         4.0|             1.0|   ANACACHO_ANA|                 RN|                26.7|  false|
|  2024-01-11|         5.0|             1.0|   ANACACHO_ANA|                 RN|               19.01|  false|
+------------+------------+----------------+---------------+-------------------+--------------------+-------+
only showi

In [0]:
spark_df.write.mode("overwrite").saveAsTable("labuser12487029_1763334363.trainDf")

In [0]:
%sql
select * from dbacademy.labuser12487029_1763334363.traindf limit 10

deliveryDate,deliveryHour,deliveryInterval,settlementPoint,settlementPointType,settlementPointPrice,DSTFlag
2024-01-11,1.0,1.0,ANACACHO_ANA,RN,142.36,false
2024-01-11,2.0,1.0,ANACACHO_ANA,RN,243.04,false
2024-01-11,3.0,1.0,ANACACHO_ANA,RN,89.6,false
2024-01-11,4.0,1.0,ANACACHO_ANA,RN,26.7,false
2024-01-11,5.0,1.0,ANACACHO_ANA,RN,19.01,false
2024-01-11,6.0,1.0,ANACACHO_ANA,RN,17.08,false
2024-01-11,7.0,1.0,ANACACHO_ANA,RN,23.11,false
2024-01-11,8.0,1.0,ANACACHO_ANA,RN,18.03,false
2024-01-11,9.0,1.0,ANACACHO_ANA,RN,17.79,false
2024-01-11,10.0,1.0,ANACACHO_ANA,RN,12.15,false
